# Common Utilities

> Fill in a module description here

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import gc

import pandas as pd
import torch
# from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from numerize import numerize

In [ ]:
#| export
# whisper_norm = BasicTextNormalizer()

In [ ]:
#| export
def is_target_text_in_range(ref):
    if ref.strip() == "ignore time segment in scoring":
        return False
    else:
        return ref.strip() != ""


def get_text(sample):
    if "text" in sample:
        return sample["text"]
    elif "sentence" in sample:
        return sample["sentence"]
    elif "normalized_text" in sample:
        return sample["normalized_text"]
    elif "transcript" in sample:
        return sample["transcript"]
    elif "transcription" in sample:
        return sample["transcription"]
    else:
        raise ValueError(
            f"Expected transcript column of either 'text', 'sentence', 'normalized_text' or 'transcript'. Got sample of "
            ".join{sample.keys()}. Ensure a text column name is present in the dataset."
        )
        
# def normalise(batch):
#     batch["norm_text"] = whisper_norm(get_text(batch))
#     return batch


def data(dataset):
    for i, item in enumerate(dataset):
        yield {**item["audio"], "reference": item["norm_text"]}

In [ ]:
#| export
def get_model_size(model):
    total_params = sum(param.numel() for param in model.parameters())
    return numerize.numerize(total_params)

In [ ]:
#| export

def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
#| export

def store_results_as_dataset(predictions, predictions_raw, references, reference_raw, model_name, time, model_size, wer, cer,saving_name):
    df = pd.DataFrame({"predictions": predictions, "ground_truth": references, "prediction_raw": predictions_raw, "references_raw": references_raw, "model_name": model_name})
    df["wer"] = df.apply(lambda row: wer(row["ground_truth"], row["predictions"]), axis=1)
    df["cer"] = df.apply(lambda row: cer(row["ground_truth"], row["predictions"]), axis=1)

    df["total_time"] = time
    df["model_size"] = model_size
    df["total_wer"] = wer
    df["total_cer"] = cer

    save_name = model_name.split("/")
    save_name += saving_name
    df.to_parquet(save_name)
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()